This notebook shows that how square wave and sin wave embedder works with different number of samples when embedding dimension is large enough.

In [ ]:
from pylab import *
from skimage.color import rgb2gray
from skimage.transform import rescale,resize
from torch.autograd import Variable
import torch, time,os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy


# Set the CUDA flag
cuda_flag = True
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
criterion = torch.nn.MSELoss()
epochs = 4000
lr = 1e-3


sample_N = 512
sample_ds = [1,2,4,8,16]
fs = [1,5,10,20,50,100,200]
embedding_dimension = 10000

Nf = len(fs)
Nd = len(sample_ds)
N_test = 1
N_repeat = 1

#kk = torch.randint(512,(N_test,))
kk = [158, 329,  47, 344, 257, 410, 426, 199, 426, 295]
#kk = [ 88, 269, 409,  59, 245, 382, 277, 362,  23, 308, 504, 448, 411, 304, 365, 200,  92, 350, 405,  77]

losses_ = np.zeros((Nd,Nf,N_test,N_repeat,epochs))
sr_ = np.zeros((Nd,Nf))
trn_psnr_ = np.zeros((Nd,Nf,N_test,N_repeat,epochs))
tst_psnr_ = np.zeros((Nd,Nf,N_test,N_repeat,epochs))

file_path = 'Final_2one/'

In [ ]:
rgb = imread('4.2.07.tiff');im = rgb2gray(rgb);
signal = torch.from_numpy(im).type(torch.FloatTensor)

In [ ]:
width = 256

# Here define the MLP
class SLP(nn.Module):
    def __init__(self,input_dim):
        super(SLP, self).__init__()
        self.fco = nn.Linear(input_dim, 1)
    def forward(self, x):
        out = self.fco(x)
        return out
    def name(self):
        return "SLP"

In [ ]:
def srank_func(X):
    # Get the SVD
    (u,s,v) = torch.svd(X)
    sr = (s*s).sum()/s[0]/s[0]
    return sr

def psnr_func(x,y):
    diff = x - y
    err = (diff*diff).flatten().mean().log10()
    return -10*err

def impulse_embedding(x, b, s=1e-4):
    emb = (abs(x-b)<s).type(torch.FloatTensor)
    return emb/emb.norm(dim=1).max()

def square_embedding(x, b, f=1):
    emb = (torch.sin(2.*np.pi*f*(x-b))>0).type(torch.FloatTensor)
    return emb/emb.norm(dim=1).max()

def shift_embedding(x, c):
    all_emb = scipy.linalg.circulant(c)
    idx = (x*all_emb.shape[0]).int().squeeze()
    emb = torch.from_numpy(all_emb[idx]).type(torch.FloatTensor)
    return emb/emb.norm(dim=1).max()
    

def norm_func(x):
    # Flatten the data 
    #x = x.flatten(1,3)
    
    # Normalize for gain and bias
    y = x-x.mean(1).unsqueeze(-1)
    y = x/x.norm(dim=1).unsqueeze(-1)    
    return y

# square wave embedding

In [ ]:
all_data = torch.from_numpy(np.linspace(0, 1, sample_N+1)[:-1].reshape(-1,1)).type(torch.FloatTensor)


for i in range(Nd):
    sample_d = sample_ds[i]
    
    for ff in range(Nf):
        f0 = fs[ff]
        
        c = np.sin(2.*np.pi*f0*np.linspace(0, 1, embedding_dimension+1)[:-1])>0
        c = c/np.linalg.norm(c)
        
        train_data = shift_embedding(all_data[::2*sample_d],c)
        test_data = shift_embedding(all_data[sample_d::2*sample_d],c)

        # Calculate the input stable rank
        sr_[i,ff] = srank_func(train_data)


        for j in range(N_test):
            start_time = time.time()


            # Prepare the data
            all_target = signal[kk[j]].reshape(-1,1)
            train_label = all_target[::2*sample_d]
            test_label = all_target[sample_d::2*sample_d]

            xt, yt = train_data.cuda(),train_label.cuda()
            xt, yt = Variable(xt), Variable(yt)

            xts, yts = test_data.cuda(),test_label.cuda()


            for k in range(N_repeat):
                # Initialize classification model to learn
                model = SLP(embedding_dimension).cuda()
                # Set the optimization
                optimizer = optim.Adam(model.parameters(), lr, betas=(0.9, 0.999),weight_decay=1e-8)


                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()

                    out = model(xt)
                    loss = criterion(out, yt)

                    loss.backward()
                    losses_[i,ff,j,k,epoch] = loss.item()
                    optimizer.step()

                    model.eval()
                    trn_psnr_[i,ff,j,k,epoch] = psnr_func(model(xt),yt)
                    tst_psnr_[i,ff,j,k,epoch] = psnr_func(model(xts),yts)


                print("==>>> N: %g, F: %g, train psnr: %g--- , test psnr: %g--- , time: %g seconds ---" 
                        % (i,ff, np.mean(trn_psnr_[i,ff,j,k,-100:]),np.mean(tst_psnr_[i,ff,j,k,-100:]),time.time() - start_time))

In [ ]:
for i in range(Nf):
    figure()
    for j in range(Nd):
        plot(np.mean(trn_psnr_[j,i,:,:],(0,1)))
        plot(np.mean(tst_psnr_[j,i,:,:],(0,1)),'--')

figure()
for i in range(Nf):
    plot(sample_ds,sr_[:,i])

In [ ]:
file_name = 'square'
np.save(file_path+file_name+'_losses',losses_)
np.save(file_path+file_name+'_sr',sr_)
np.save(file_path+file_name+'_trn',trn_psnr_)
np.save(file_path+file_name+'_tst',tst_psnr_)

# sin embedding

In [ ]:
all_data = torch.from_numpy(np.linspace(0, 1, sample_N+1)[:-1].reshape(-1,1)).type(torch.FloatTensor)


for i in range(Nd):
    sample_d = sample_ds[i]
    
    for ff in range(Nf):
        f0 = fs[ff]
        
        c = np.sin(2.*np.pi*f0*np.linspace(0, 1, embedding_dimension+1)[:-1])
        c = c/np.linalg.norm(c)
        
        train_data = shift_embedding(all_data[::2*sample_d],c)
        test_data = shift_embedding(all_data[sample_d::2*sample_d],c)

        # Calculate the input stable rank
        sr_[i,ff] = srank_func(train_data)


        for j in range(N_test):
            start_time = time.time()


            # Prepare the data
            all_target = signal[kk[j]].reshape(-1,1)
            train_label = all_target[::2*sample_d]
            test_label = all_target[sample_d::2*sample_d]

            xt, yt = train_data.cuda(),train_label.cuda()
            xt, yt = Variable(xt), Variable(yt)

            xts, yts = test_data.cuda(),test_label.cuda()


            for k in range(N_repeat):
                # Initialize classification model to learn
                model = SLP(embedding_dimension).cuda()
                # Set the optimization
                optimizer = optim.Adam(model.parameters(), lr, betas=(0.9, 0.999),weight_decay=1e-8)


                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()

                    out = model(xt)
                    loss = criterion(out, yt)

                    loss.backward()
                    losses_[i,ff,j,k,epoch] = loss.item()
                    optimizer.step()

                    model.eval()
                    trn_psnr_[i,ff,j,k,epoch] = psnr_func(model(xt),yt)
                    tst_psnr_[i,ff,j,k,epoch] = psnr_func(model(xts),yts)


                print("==>>> N: %g, F: %g, train psnr: %g--- , test psnr: %g--- , time: %g seconds ---" 
                        % (i,ff, np.mean(trn_psnr_[i,ff,j,k,-100:]),np.mean(tst_psnr_[i,ff,j,k,-100:]),time.time() - start_time))

In [ ]:
for i in range(Nf):
    figure()
    for j in range(Nd):
        plot(np.mean(trn_psnr_[j,i,:,:],(0,1)))
        plot(np.mean(tst_psnr_[j,i,:,:],(0,1)),'--')

figure()
for i in range(Nf):
    plot(sample_ds,sr_[:,i])

In [ ]:
file_name = 'sin'
np.save(file_path+file_name+'_losses',losses_)
np.save(file_path+file_name+'_sr',sr_)
np.save(file_path+file_name+'_trn',trn_psnr_)
np.save(file_path+file_name+'_tst',tst_psnr_)

In [ ]:
signal = torch.randn((512,512)).type(torch.FloatTensor)

In [ ]:
from pylab import *
from skimage.color import rgb2gray
from skimage.transform import rescale,resize
from torch.autograd import Variable
import torch, time,os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import scipy


# Set the CUDA flag
cuda_flag = True
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
criterion = torch.nn.MSELoss()
epochs = 4000
lr = 1e-3


sample_N = 512
sample_ds = [1,2,3,4,6,8,11,16]
fs = [1]
embedding_dimension = 10000

Nf = len(fs)
Nd = len(sample_ds)
N_test = 1
N_repeat = 1

#kk = torch.randint(512,(N_test,))
kk = [158, 329,  47, 344, 257, 410, 426, 199, 426, 295]
#kk = [ 88, 269, 409,  59, 245, 382, 277, 362,  23, 308, 504, 448, 411, 304, 365, 200,  92, 350, 405,  77]

losses_ = np.zeros((Nd,Nf,N_test,N_repeat,epochs))
sr_ = np.zeros((Nd,Nf))
trn_psnr_ = np.zeros((Nd,Nf,N_test,N_repeat,epochs))
tst_psnr_ = np.zeros((Nd,Nf,N_test,N_repeat,epochs))

file_path = 'Final_2one/'

In [ ]:
all_data = torch.from_numpy(np.linspace(0, 1, sample_N+1)[:-1].reshape(-1,1)).type(torch.FloatTensor)


for i in range(Nd):
    sample_d = sample_ds[i]
    
    for ff in range(Nf):
        f0 = fs[ff]
        
        c = np.sin(2.*np.pi*f0*np.linspace(0, 1, embedding_dimension+1)[:-1])>0
        c = c/np.linalg.norm(c)
        
        train_data = shift_embedding(all_data[::2*sample_d],c)
        test_data = shift_embedding(all_data[sample_d::2*sample_d],c)

        # Calculate the input stable rank
        sr_[i,ff] = srank_func(train_data)


        for j in range(N_test):
            start_time = time.time()


            # Prepare the data
            all_target = signal[kk[j]].reshape(-1,1)
            train_label = all_target[::2*sample_d]
            test_label = all_target[sample_d::2*sample_d]

            xt, yt = train_data.cuda(),train_label.cuda()
            xt, yt = Variable(xt), Variable(yt)

            xts, yts = test_data.cuda(),test_label.cuda()


            for k in range(N_repeat):
                # Initialize classification model to learn
                model = SLP(embedding_dimension).cuda()
                # Set the optimization
                optimizer = optim.Adam(model.parameters(), lr, betas=(0.9, 0.999),weight_decay=1e-8)


                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()

                    out = model(xt)
                    loss = criterion(out, yt)

                    loss.backward()
                    losses_[i,ff,j,k,epoch] = loss.item()
                    optimizer.step()

                    model.eval()
                    trn_psnr_[i,ff,j,k,epoch] = psnr_func(model(xt),yt)
                    tst_psnr_[i,ff,j,k,epoch] = psnr_func(model(xts),yts)


                print("==>>> N: %g, F: %g, train psnr: %g--- , test psnr: %g--- , time: %g seconds ---" 
                        % (i,ff, np.mean(trn_psnr_[i,ff,j,k,-100:]),np.mean(tst_psnr_[i,ff,j,k,-100:]),time.time() - start_time))

In [ ]:
for i in range(Nf):
    figure()
    for j in range(Nd):
        plot(np.mean(trn_psnr_[j,i,:,:],(0,1)))
        plot(np.mean(tst_psnr_[j,i,:,:],(0,1)),'--')

figure()
for i in range(Nf):
    plot(sample_ds,sr_[:,i])

In [ ]:
file_name = 'square_random'
np.save(file_path+file_name+'_losses',losses_)
np.save(file_path+file_name+'_sr',sr_)
np.save(file_path+file_name+'_trn',trn_psnr_)
np.save(file_path+file_name+'_tst',tst_psnr_)

In [ ]:
all_data = torch.from_numpy(np.linspace(0, 1, sample_N+1)[:-1].reshape(-1,1)).type(torch.FloatTensor)


for i in range(Nd):
    sample_d = sample_ds[i]
    
    for ff in range(Nf):
        f0 = fs[ff]
        
        c = np.sin(2.*np.pi*f0*np.linspace(0, 1, embedding_dimension+1)[:-1])
        c = c/np.linalg.norm(c)
        
        train_data = shift_embedding(all_data[::2*sample_d],c)
        test_data = shift_embedding(all_data[sample_d::2*sample_d],c)

        # Calculate the input stable rank
        sr_[i,ff] = srank_func(train_data)


        for j in range(N_test):
            start_time = time.time()


            # Prepare the data
            all_target = signal[kk[j]].reshape(-1,1)
            train_label = all_target[::2*sample_d]
            test_label = all_target[sample_d::2*sample_d]

            xt, yt = train_data.cuda(),train_label.cuda()
            xt, yt = Variable(xt), Variable(yt)

            xts, yts = test_data.cuda(),test_label.cuda()


            for k in range(N_repeat):
                # Initialize classification model to learn
                model = SLP(embedding_dimension).cuda()
                # Set the optimization
                optimizer = optim.Adam(model.parameters(), lr, betas=(0.9, 0.999),weight_decay=1e-8)


                for epoch in range(epochs):
                    model.train()
                    optimizer.zero_grad()

                    out = model(xt)
                    loss = criterion(out, yt)

                    loss.backward()
                    losses_[i,ff,j,k,epoch] = loss.item()
                    optimizer.step()

                    model.eval()
                    trn_psnr_[i,ff,j,k,epoch] = psnr_func(model(xt),yt)
                    tst_psnr_[i,ff,j,k,epoch] = psnr_func(model(xts),yts)


                print("==>>> N: %g, F: %g, train psnr: %g--- , test psnr: %g--- , time: %g seconds ---" 
                        % (i,ff, np.mean(trn_psnr_[i,ff,j,k,-100:]),np.mean(tst_psnr_[i,ff,j,k,-100:]),time.time() - start_time))

In [ ]:
for i in range(Nf):
    figure()
    for j in range(Nd):
        plot(np.mean(trn_psnr_[j,i,:,:],(0,1)))
        plot(np.mean(tst_psnr_[j,i,:,:],(0,1)),'--')

figure()
for i in range(Nf):
    plot(sample_ds,sr_[:,i])

In [ ]:
file_name = 'sin_random'
np.save(file_path+file_name+'_losses',losses_)
np.save(file_path+file_name+'_sr',sr_)
np.save(file_path+file_name+'_trn',trn_psnr_)
np.save(file_path+file_name+'_tst',tst_psnr_)